<a href="https://colab.research.google.com/github/rahatarinasir/Dayche/blob/master/Logistic_Regression_with_PySpark_Chrun_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 199 kB 48.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=64ad3d612c8306d1e4d4864335ae42977a7164df7c5f9bb306c4286b94c99478
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("churn_pred").getOrCreate()

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/PySpark/customer_churn.csv",header=True , inferSchema=True)

In [ ]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [ ]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [ ]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [ ]:
df.dtypes

[('Names', 'string'),
 ('Age', 'double'),
 ('Total_Purchase', 'double'),
 ('Account_Manager', 'int'),
 ('Years', 'double'),
 ('Num_Sites', 'double'),
 ('Onboard_date', 'timestamp'),
 ('Location', 'string'),
 ('Company', 'string'),
 ('Churn', 'int')]

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=[
#  'Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
#  'Onboard_date',
#  'Location',
#  'Company',
#  'Churn'
 ],
 outputCol="features")

In [ ]:
output = assembler.transform(df)

In [ ]:
df_final = output.select("features","churn")

In [ ]:
df_final.show()

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
train , test = df_final.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression(labelCol="churn")

In [ ]:
lrm = lr.fit(train)

In [ ]:
lrm.summary

In [ ]:
lrm.summary.accuracy

0.9008264462809917

In [ ]:
lrm.summary.areaUnderROC

0.9112879457707043

In [ ]:
lrm.summary.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|  0.0|[5.08099498428111...|[0.99382464854368...|       0.0|
|[25.0,9672.03,0.0...|  0.0|[5.02688685077914...|[0.99348354414542...|       0.0|
|[26.0,8787.39,1.0...|  1.0|[0.95504124691572...|[0.72212788249845...|       0.0|
|[28.0,8670.98,0.0...|  0.0|[8.18523192302290...|[0.99972133790075...|       0.0|
|[28.0,9090.43,1.0...|  0.0|[1.81534695451316...|[0.86000686119937...|       0.0|
|[28.0,11128.95,1....|  0.0|[4.54746566713651...|[0.98951703754350...|       0.0|
|[28.0,11204.23,0....|  0.0|[2.22297822921711...|[0.90229406960740...|       0.0|
|[28.0,11245.38,0....|  0.0|[3.99922957358199...|[0.98200017716787...|       0.0|
|[29.0,5900.78,1.0...|  0.0|[4.39301705330870...|[0.98778761432430...|       0.0|
|[29.0,8688.17,1

In [ ]:
lrm.summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                726|                726|
|   mean|0.16115702479338842|0.12534435261707988|
| stddev| 0.3679292062699649| 0.3313372360848368|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
pred_labels = lrm.evaluate(test)

In [ ]:
pred_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8939.61,0.0...|    0|[6.77319428533299...|[0.99885727288346...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.81400928727858...|[0.99702344972921...|       0.0|
|[29.0,13255.05,1....|    0|[4.53822265133025...|[0.98942072399806...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.55310942492720...|[0.97216170126281...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.31631382154242...|[0.99819666722117...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.36480403632186...|[0.98744254793424...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.43245675288004...|[0.96870363471561...|       0.0|
|[30.0,10183.98,1....|    0|[3.20903519234107...|[0.96117287534045...|       0.0|
|[30.0,13473.35,0....|    0|[3.10432436302006...|[0.95707076878213...|       0.0|
|[31.0,10182.6,1

In [ ]:
eval = BinaryClassificationEvaluator(rawPredictionCol="prediction" , labelCol='churn')

In [ ]:
auc = eval.evaluate(pred_labels.predictions)

In [ ]:
auc

0.6827852998065764